## 조달청 나라장터 종합쇼핑몰 추천시스템 개발_API 구현

### 01. 조달정보개방포털 '종합쇼핑몰 품목 등록 내역'

In [7]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import pandas as pd
import xmltodict
import json

key='hnpcDKStTuq1euifHeyFy05Zc7DhedODjjRXadm%2BbbG4%2Fs6m0wWUps1xCOg1L%2BacMTuaWWNX658YYydKvWrS8w%3D%3D'
url = f'http://apis.data.go.kr/1230000/ShoppingMallPrdctInfoService04/getDlvrReqDtlInfoList?serviceKey={key}&'

queryParams = urlencode({ quote_plus('pageNo') : 1, 
                          quote_plus('numOfRows') : 999, # 최대트래픽 수 제한으로 인한 최대 열 개수 설정
                          quote_plus('type') : 'xml',
                          quote_plus('inqryDiv') : 1,
                          quote_plus('inqryBgnDate') : '20220103',
                          quote_plus('inqryEndDate') : '20220203', # 데이터 수집 기간 한달로 설정
                          quote_plus('dtilPrdctClsfcNoNm') : '교구'}) 
url2 = url + queryParams

response = urlopen(url2)

results = response.read().decode("utf-8")
results_to_json = xmltodict.parse(results)
data = json.loads(json.dumps(results_to_json))
print(type(data))   # dic
# print(data)

<class 'dict'>


In [8]:
history = data['response']['body']['items']['item']

#추가하고 싶은 리스트 생성
col_1=[]
col_2=[]
col_3=[]
col_4=[]
col_5=[]
col_6=[]

for i in history:
    try:
        col_1.append(i["dtilPrdctClsfcNoNm"]) # 세부품명
        col_2.append(i["prdctIdntNo"]) # 물품식별번호
        col_3.append(i["prdctIdntNoNm"]) # 품목
        col_4.append(i["fnlDlvrReqYn"]) # 최종납품등록여부
        col_5.append(i["dminsttCd"]) # 수요기관코드
        col_6.append(i["dminsttNm"]) # 수요기관명
    except:
        continue   

In [9]:
history = pd.DataFrame([col_1,col_2,col_3,col_4,col_5,col_6]).T
history.columns=['세부품명','물품식별번호','품목','최종납품등록여부','수요기관코드','수요기관명'] 
history = history.sort_values(by='세부품명', ascending=True)

# history.head()
history.info()
# df.describe()

# csv 파일 생성
history.to_csv('shopping_history.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 470 entries, 0 to 125
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   세부품명      470 non-null    object
 1   물품식별번호    470 non-null    object
 2   품목        470 non-null    object
 3   최종납품등록여부  470 non-null    object
 4   수요기관코드    470 non-null    object
 5   수요기관명     470 non-null    object
dtypes: object(6)
memory usage: 25.7+ KB


### 02. 조달정보개방포털 '종합쇼핑몰 납품요구 상세내역'

In [10]:
key='hnpcDKStTuq1euifHeyFy05Zc7DhedODjjRXadm%2BbbG4%2Fs6m0wWUps1xCOg1L%2BacMTuaWWNX658YYydKvWrS8w%3D%3D'
url = f'http://apis.data.go.kr/1230000/ShoppingMallPrdctInfoService04/getShoppingMallPrdctInfoList?serviceKey={key}&'
queryParams = urlencode({ quote_plus('pageNo') : 1, 
                          quote_plus('numOfRows') : 100, # 개인 장비의 저장공간 한계로 인한 열 개수 설정
                          quote_plus('type') : 'xml',
                          quote_plus('inqryDiv') : 1,
                          quote_plus('inqryBgnDate') : '20210203',
                          quote_plus('inqryEndDate') : '20220203', # 데이터 수집 기간 1년으로 설정
                          quote_plus('dtilPrdctClsfcNoNm') : '교구',
                          quote_plus('regtCncelYn') : 'N'})  # 현재 등록된 상품 확인을 위함
url2 = url + queryParams

response = urlopen(url2)

results = response.read().decode("utf-8")
results_to_json = xmltodict.parse(results)
data = json.loads(json.dumps(results_to_json))
print(type(data))   # dic
# print(data)

<class 'dict'>


In [11]:
features = data['response']['body']['items']['item']

#추가하고 싶은 리스트 생성
col_1=[]
col_2=[]
col_3=[]
col_4=[]

for i in features:
    try:
        col_1.append(i["dtilPrdctClsfcNoNm"]) # 세부품명
        col_2.append(i["prdctIdntNo"]) # 물품식별번호
        col_3.append(i["prdctSpecNm"]) # 품목
        col_4.append(i["prodctCertList"]) # 제품인증목록

    except:
        continue   

In [12]:
features = pd.DataFrame([col_1,col_2,col_3,col_4]).T
features.columns=['세부품명','물품식별번호','품목','제품인증목록'] 
features = features.sort_values(by='세부품명', ascending=True)

features.head()
# df.info()
# df.describe()

# csv 파일 생성
features.to_csv('shopping_feature.csv')